<a href="https://colab.research.google.com/github/laresamdeola/Computer-Vision/blob/main/Naira_Currency_Counterfeit_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [37]:
tf.__version__

'2.15.0'

In [38]:
import os
import shutil

In [39]:
os.getcwd()

'/content'

In [40]:
os.listdir('NCC_data')

['train_data', 'test_data']

In [41]:
# shutil.rmtree('NCC_data/.ipynb_checkpoints')

In [42]:
os.listdir('NCC_data')

['train_data', 'test_data']

In [43]:
os.listdir('NCC_data/train_data')

['others', 'naira']

In [44]:
os.listdir('NCC_data/test_data')

['others', 'naira']

In [45]:
os.listdir('NCC_data/train_data')

['others', 'naira']

In [46]:
os.listdir('NCC_data/test_data')

['others', 'naira']

In [47]:
# shutil.rmtree('NCC_data/train_data/.ipynb_checkpoints')

In [48]:
# shutil.rmtree('NCC_data/test_data/.ipynb_checkpoints')

In [49]:
os.listdir('NCC_data/train_data')
os.listdir('NCC_data/test_data')

['others', 'naira']

In [50]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_set = train_datagen.flow_from_directory(
    'NCC_data/train_data',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 36 images belonging to 2 classes.


In [51]:
print(train_set.labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [81]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_set = test_datagen.flow_from_directory(
    'NCC_data/test_data',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 9 images belonging to 2 classes.


In [82]:
print(test_set.labels)

[0 0 0 0 0 1 1 1 1]


In [53]:
cnn = tf.keras.models.Sequential()

In [54]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=3,
                               activation='relu',
                               input_shape=[64,64,3]))

In [55]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [56]:
cnn.add(tf.keras.layers.Flatten())

In [57]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [58]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [59]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [60]:
cnn.fit(x=train_set, validation_data=test_set, epochs=60)

Epoch 1/60
2/2 [==============================] - 2s 346ms/step - loss: 9.0261 - accuracy: 0.4722 - val_loss: 3.5782 - val_accuracy: 0.5000
Epoch 2/60
2/2 [==============================] - 0s 148ms/step - loss: 3.3889 - accuracy: 0.5000 - val_loss: 2.6891 - val_accuracy: 0.5000
Epoch 3/60
2/2 [==============================] - 0s 424ms/step - loss: 1.9578 - accuracy: 0.5000 - val_loss: 1.4476 - val_accuracy: 0.5000
Epoch 4/60
2/2 [==============================] - 0s 424ms/step - loss: 0.7929 - accuracy: 0.5000 - val_loss: 0.6839 - val_accuracy: 0.6000
Epoch 5/60
2/2 [==============================] - 0s 172ms/step - loss: 0.6926 - accuracy: 0.7222 - val_loss: 0.7049 - val_accuracy: 0.5000
Epoch 6/60
2/2 [==============================] - 0s 424ms/step - loss: 0.7230 - accuracy: 0.5000 - val_loss: 0.7255 - val_accuracy: 0.5000
Epoch 7/60
2/2 [==============================] - 0s 151ms/step - loss: 0.7324 - accuracy: 0.5000 - val_loss: 0.6783 - val_accuracy: 0.6000
Epoch 8/60
2/2 [====

In [61]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 30752)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               3936384   
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3937409 (15.02 MB)
Trainable params: 3937409 (15.02 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [64]:
cnn.save(f'{os.getcwd()}/NCC_data/ncc_model.h5')

In [72]:
def preprocess_image(path):
  img_path = image.load_img(path, target_size=(64, 64))
  img_array = image.img_to_array(img_path)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = img_array/255.0
  return img_array

In [83]:
predictions = cnn.predict(preprocess_image('/content/NCC_data/test_data/naira/1000 naira back 2.PNG'))

1/1 [==============================] - 0s 27ms/step


In [84]:
# Since you're using sigmoid activation, predictions will be probabilities
# Assuming a binary classification task

# LABELS - naira == 0, others == 1

import math

if predictions[0] > 0.5:
    predicted_class = 1  # Class 1
else:
    predicted_class = 0  # Class 0

confidence = predictions[0]

print(f"Predicted Class: {predicted_class}")
print(f"Confidence: {confidence}")


Predicted Class: 0
Confidence: [0.36505264]


In [63]:
# Credits:

# Lare Adeola (2024)
# Naira Currency Counterfeit Model.
